In [5]:
import re
import time
import urllib3
import requests
import pandas as pd
from bs4 import BeautifulSoup
from collections import defaultdict

urllib3.disable_warnings() # Fix the warnings!
from collections import Counter
import nltk
import string
from nltk.tokenize import word_tokenize

In [40]:
base = 'https://scholarshipdb.net/scholarships-in-United-States/Ph-D-Assistantship-Climate-Change-Responses-In-Coastal-Arctic-Wetlands-Utah-State-University=NhoiTd9j7RGUYQAlkGUTnw.html'
html_text = requests.get(base, verify=False)
soup = BeautifulSoup(html_text.content, "html.parser")

In [41]:
for child in soup.ul.children:
    print(child)



<li><a data-ev-action="PhD" data-ev-cat="Nav" href="/PhD-scholarships">PhD</a></li>


<li><a data-ev-action="Postdoctoral" data-ev-cat="Nav" href="/Postdoctoral-scholarships">Postdoctoral</a></li>


<li><a data-ev-action="Master" data-ev-cat="Nav" href="/scholarships/Program-Master">Master</a></li>


<li><a data-ev-action="Undergraduate" data-ev-cat="Nav" href="/scholarships/Program-Undergraduate">Undergraduate</a></li>


<li>
<a data-ev-action="Featured" data-ev-cat="Nav" href="/scholarships/Tags-Featured">
<i class="glyphicon glyphicon-star red"></i> Featured
                </a>
</li>


<li><a data-ev-action="Universities" data-ev-cat="Nav" href="/universities">Universities</a></li>


<li><a data-ev-action="Blogs" data-ev-cat="Nav" href="/blogs">Blogs</a></li>


<li><a data-ev-action="Browse" data-ev-cat="Nav" href="/scholarships-browse">Browse</a></li>




In [45]:
allPhD = defaultdict(list)
allPhD['Title'].append(soup.find('h1').find('a').text)

In [47]:
allPhD['location'].append(soup.find('h2').find('a').text)

In [61]:
#Useful for uni, location, department, country
list_meta = soup.find_all(attrs={'class':'text-success'})
[i.text for i in list_meta]

['United States',
 'Utah State University, Dept. of Geosciences',
 'United States',
 'Utah State University (Department of Civil and Environmental Engineering)',
 'United States']

In [68]:
list_describe = soup.find_all(attrs={'class':'description'})

In [78]:
description = soup.find(attrs={'class':'description'})

In [112]:
help(description.get_text)

Help on method get_text in module bs4.element:

get_text(separator='', strip=False, types=<object object at 0x000001C09A636630>) method of bs4.element.Tag instance
    Get all child strings of this PageElement, concatenated using the
    given separator.
    
    :param separator: Strings will be concatenated using this separator.
    
    :param strip: If True, strings will be stripped before being
        concatenated.
    
    :param types: A tuple of NavigableString subclasses. Any
        strings of a subclass not found in this list will be
        ignored. Although there are exceptions, the default
        behavior in most cases is to consider only NavigableString
        and CData objects. That means no comments, processing
        instructions, etc.
    
    :return: A string.



In [144]:
description_df = re.split(r'<dt>|</dt>',str(description))

In [149]:
description_df

['<div class="description">\n<br/>Click for a hub of Extension resources related to the current COVID-19 situation.<br/><dl>',
 'Agency',
 '<dd>Utah State University</dd>',
 'Location',
 '<dd>Logan, UT; YK Delta, Alaska</dd>',
 'Job Category',
 '<dd>Graduate Assistantships</dd>',
 'Salary',
 '<dd>$24,000/yr</dd>',
 'Start Date',
 '<dd>05/01/2023</dd>',
 'Last Date to Apply',
 '<dd>12/15/2022</dd>',
 'Website',
 '<dd>https://karenhbeard.webs.com/ </dd>',
 'Description',
 '<dd>A PhD student position is available to participate in an NSF-funded project examining the interaction of multiple climate change forcings on vegetation and ecosystem functioning in Arctic wetlands through field and laboratory research. Field work is conducted in the Yukon-Kuskokwim Delta in Alaska and will involve an experiment with flooding, warming, and herbivory treatments followed by measurements of vegetation responses, and CO2 and CH4 gas fluxes. The successful candidate must start no later than 1 May 2023 (a

In [151]:
University_name = description_df[2]

In [154]:
Location = description_df[4]

In [157]:
Job_category = description_df[6]

In [159]:
Salary = description_df[8]

In [161]:
Start_date = description_df[10]

In [167]:
Description = description_df[16]

In [142]:
description_df = ' '.join(re.split(r'<..>|<...>', str(description)))

In [143]:
description_df

'<div class="description">\n Click for a hub of Extension resources related to the current COVID-19 situation.   Agency  Utah State University  Location  Logan, UT; YK Delta, Alaska  Job Category  Graduate Assistantships  Salary  $24,000/yr  Start Date  05/01/2023  Last Date to Apply  12/15/2022  Website  https://karenhbeard.webs.com/   Description  A PhD student position is available to participate in an NSF-funded project examining the interaction of multiple climate change forcings on vegetation and ecosystem functioning in Arctic wetlands through field and laboratory research. Field work is conducted in the Yukon-Kuskokwim Delta in Alaska and will involve an experiment with flooding, warming, and herbivory treatments followed by measurements of vegetation responses, and CO2 and CH4 gas fluxes. The successful candidate must start no later than 1 May 2023 (although 1 April 2023 is preferred), and be willing to work 3 months each summer in an extremely remote setting in western Alaska

In [140]:
# Create a parser instance able to parse invalid markup.
links = soup.select(".list-unstyled h4 a")
if len(links) == 0:
    break
for link in links:
    link = f'https://scholarshipdb.net{link.get("href")}'
    html = requests.get(link, verify=False)
    soup2 = BeautifulSoup(html.text, 'html.parser')
    allPhD['title'].append(soup2.find('head').title.text)
    allPhD['location'].append(soup2.find('h2').find('a').text)
    allPhD['keywords'].append(soup2.find('meta',attrs={"name": 'keywords'})['content'])
    allPhD['meta'].append(soup2.find('meta',attrs={"name": 'description'})['content'])
    allPhD['description'].append(soup2.find('div', attrs={'class': 'description'}).text.strip())
    time.sleep(5)
    page += 1
allPhD = pd.DataFrame(allPhD)
allPhD.to_pickle(f'scraping_scholarshipDB_dataframe-{startPage}-{page}.pkl')

SyntaxError: 'break' outside loop (3658247088.py, line 4)

In [ ]:
# Make lower case, remove non-useful chars
description_df = re.sub(r'[^a-z0-9/:.,$@]+', ' ', description_df.lower())
# Remove multiple spaces
description_df = ' '.join(description_df.split())